### Part - 1

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

commits_csv = "diffanalysis.csv"
df = pd.read_csv(commits_csv)
model = SentenceTransformer('microsoft/codebert-base')

def compute_similarity(code_diff, commit_msg):
    if pd.isna(code_diff) or pd.isna(commit_msg):
        return 0
    embeddings = model.encode([code_diff, commit_msg], convert_to_tensor=True)
    cos_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return (cos_sim + 1) / 2 

print(df.columns)

df['similarity'] = df.apply(lambda row: compute_similarity(row['Diff'], row['Message']), axis=1)

output_path = "/Users/tejasmacipad/Desktop/Third_year/STT/lab2/similarity_codebert.csv"
df.to_csv(output_path, index=False)

print("Similarities computed and saved to:", output_path)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.


Index(['Hash', 'Message', 'Filename', 'Source Code (prev)',
       'Source Code (current)', 'Diff', 'LLM Inference', 'rectified message'],
      dtype='object')
Similarities computed and saved to: /Users/tejasmacipad/Desktop/Third_year/STT/lab2/similarity_codebert.csv


### Part - II

In [16]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

file_path = "diffanalysis.csv"
df = pd.read_csv(file_path)

model = SentenceTransformer('microsoft/codebert-base')

def compute_similarity(code_diff, commit_msg):
    if pd.isna(code_diff) or pd.isna(commit_msg):
        return 0
    embeddings = model.encode([code_diff, commit_msg], convert_to_tensor=True)
    cos_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return (cos_sim + 1) / 2

df['similarity'] = df.apply(lambda row: compute_similarity(row['Diff'], row['LLM Inference']), axis=1)

THRESHOLD = 0.9
df['hit'] = df['similarity'] >= THRESHOLD
hit_rate = df['hit'].sum() / len(df) * 100

average_similarity = df['similarity'].mean()
total_commits = len(df)

print(f"Total commits: {total_commits}")
print(f"Average cosine similarity: {average_similarity:.4f}")
print(f"Hit rate (threshold {THRESHOLD}): {hit_rate:.2f}%")

output_path = "/Users/tejasmacipad/Desktop/Third_year/STT/lab2/newmodelfile_with_similarity_codebert.csv"
df.to_csv(output_path, index=False)
print("Saved CSV with similarity scores to:", output_path)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.


Total commits: 727
Average cosine similarity: 0.9239
Hit rate (threshold 0.9): 88.72%
Saved CSV with similarity scores to: /Users/tejasmacipad/Desktop/Third_year/STT/lab2/newmodelfile_with_similarity_codebert.csv


## Part - III

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

file_path = "diffanalysis.csv"
df = pd.read_csv(file_path)

model = SentenceTransformer("microsoft/codebert-base")

def compute_similarity(code_diff, commit_msg):
    if pd.isna(code_diff) or pd.isna(commit_msg):
        return 0
    embeddings = model.encode([code_diff, commit_msg], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return (similarity + 1) / 2

df['sim_original'] = df.apply(lambda row: compute_similarity(row['Diff'], row['Message']), axis=1)
df['sim_rectified'] = df.apply(lambda row: compute_similarity(row['Diff'], row['rectified message']), axis=1)

df['improvement'] = df['sim_rectified'] - df['sim_original']
average_improvement = df['improvement'].mean()

SIMILARITY_THRESHOLD = 0.9
num_hits = (df['sim_rectified'] >= SIMILARITY_THRESHOLD).sum()
total_commits = len(df)
hit_rate = num_hits / total_commits

print(f"Average rectification improvement: {average_improvement:.4f}")
print(f"Total commits: {total_commits}")
print(f"Commits above threshold ({SIMILARITY_THRESHOLD}): {num_hits}")
print(f"Hit rate: {hit_rate:.2%}")
print(f"recification similarity: {df["sim_rectified"].mean()}")

output_path = "/Users/tejasmacipad/Desktop/Third_year/STT/lab2/newmodelfile_with_rectification_codebert.csv"
df.to_csv(output_path, index=False)
print("Saved CSV with rectification and hit rate to:", output_path)

Average rectification improvement: 0.0171
Total commits: 727
Commits above threshold (0.9): 668
Hit rate: 91.88%
recification similarity: 0.9307055559414125
Saved CSV with rectification and hit rate to: /Users/tejasmacipad/Desktop/Third_year/STT/lab2/newmodelfile_with_rectification_codebert.csv
